In [ ]:
import numpy as np
import tensorflow as tf
import cv2
from tqdm import tqdm
import os

# Loading TFLite model
interpreter = tf.lite.Interpreter(model_path="/content/u2net_quant_float16.tflite")
interpreter.allocate_tensors()

# Getting input & output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# For float16 model, make sure input is float32 (TFLite handles conversion internally)
input_shape = input_details[0]['shape']


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, jaccard_score

In [2]:
from PIL import Image
import matplotlib.pyplot as plt

Testing Accuracy


In [3]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="pOcVYS0YKYre27bqF1it")
project = rf.workspace("uidaiproject").project("fingertip-segmentation")
version = project.version(5)
dataset = version.download("coco-segmentation")


loading Roboflow workspace...
loading Roboflow project...


In [4]:
print(dataset.location)

/content/fingertip-segmentation-5


In [ ]:
import os
from pycocotools.coco import COCO
import numpy as np
from PIL import Image

def coco_to_masks(json_path, img_dir, mask_dir):
    os.makedirs(mask_dir, exist_ok=True)
    coco = COCO(json_path)

    for img in coco.imgs.values():
        img_id = img['id']
        img_name = img['file_name']
        h, w = img['height'], img['width']
        ann_ids = coco.getAnnIds(imgIds=img_id)
        anns = coco.loadAnns(ann_ids)

        mask = np.zeros((h, w), dtype=np.uint8)
        for ann in anns:
            m = coco.annToMask(ann)
            mask = np.maximum(mask, m * 255)

        mask_img = Image.fromarray(mask)
        mask_img.save(os.path.join(mask_dir, os.path.splitext(img_name)[0] + ".png"))

# Converting masks
base_dir = dataset.location

coco_to_masks(
    os.path.join(base_dir, "train/_annotations.coco.json"),
    os.path.join(base_dir, "train/images"),
    os.path.join(base_dir, "train/masks")
)

coco_to_masks(
    os.path.join(base_dir, "valid/_annotations.coco.json"),
    os.path.join(base_dir, "valid/images"),
    os.path.join(base_dir, "valid/masks")
)

coco_to_masks(
    os.path.join(base_dir, "test/_annotations.coco.json"),
    os.path.join(base_dir, "test/images"),
    os.path.join(base_dir, "test/masks")
)

loading annotations into memory...
Done (t=0.04s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [ ]:
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (320, 320))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image.astype(np.float32) / 255.0
    image = np.expand_dims(image, axis=0)  # Adding batch dimension
    return image


In [ ]:
def predict_mask_tflite(image_path):
    image = preprocess_image(image_path)

    interpreter.set_tensor(input_details[0]['index'], image)
    interpreter.invoke()

    output = interpreter.get_tensor(output_details[0]['index'])[0, :, :, 0]  # Removing batch and channel dim
    pred_mask = (output > 0.5).astype(np.uint8)  # Binarizing

    return pred_mask


In [ ]:
def evaluate_tflite_on_dataset(image_dir, mask_dir):
    image_files = sorted([f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png', '.jpeg'))])

    all_preds = []
    all_gts = []

    for file in tqdm(image_files):
        img_path = os.path.join(image_dir, file)
        mask_path = os.path.join(mask_dir, file.replace('.jpg', '.png').replace('.jpeg', '.png'))

        # Get predicted mask
        pred_mask = predict_mask_tflite(img_path).flatten()

        # Load ground truth mask
        gt_mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        gt_mask = cv2.resize(gt_mask, (320, 320))
        gt_mask = (gt_mask > 127).astype(np.uint8).flatten()

        all_preds.extend(pred_mask)
        all_gts.extend(gt_mask)

    print("Accuracy :", accuracy_score(all_gts, all_preds))
    print("Precision:", precision_score(all_gts, all_preds))
    print("Recall   :", recall_score(all_gts, all_preds))
    print("F1 Score :", f1_score(all_gts, all_preds))
    print("IoU      :", jaccard_score(all_gts, all_preds))


In [9]:
evaluate_tflite_on_dataset("/content/fingertip-segmentation-5/test", "/content/fingertip-segmentation-5/test/masks")


100%|██████████| 126/126 [01:51<00:00,  1.13it/s]


Accuracy : 0.9525862630208334
Precision: 0.9132263352753563
Recall   : 0.8952033004019362
F1 Score : 0.9041250079340638
IoU      : 0.8250256776364733
